In [6]:
from hdfs import InsecureClient
from pyspark.sql import SparkSession

In [2]:
client = InsecureClient("http://hadoop:50070",user="nifi")

In [4]:
client.list('/dfs-data')

['1899eaf1-5a19-4143-b86b-f589b7e1bee9',
 '2b69774f-fcd0-417b-8f7d-47306da299fd',
 '2e490659-c58a-41c8-8a02-ec3f72c6c7c8',
 '4c008725-40f1-4626-97d2-dc134f2577b9',
 '776c5a2f-a0e4-4d81-97a9-2c1adab66a96',
 '9496bdb8-a7ca-4f69-8fea-81e1017780eb',
 'b01c92bf-2452-4bc7-9b4b-f8bd52bee68c',
 'c66a9d88-4b6f-41c3-a84d-fbedcd545299',
 'f9242c54-209e-4d88-bd1a-610628111851',
 'f9c61f19-f01f-4764-b920-136fb223d988']

In [7]:
spark = (SparkSession
        .builder
        .appName("hdfsprocessor")
        .getOrCreate())

In [10]:
df_load = spark.read.format("csv").option("delimiter", ",").option("header", "false").load(
    "hdfs://hadoop:8020/dfs-data/1899eaf1-5a19-4143-b86b-f589b7e1bee9")

In [11]:
df_load.show(5)

+-------+--------------------+-----+--------------------+-----------+-----+----------+-----------+---+---+----+----+----+----+----+----+--------------------+------------+--------------------+--------+-----------+-----------+-----+----------+-----------+----+----+----+----+----+----+----+----+-----+--------------------+--------------------+--------+-----------+-----------+-----+----------+-----------+----+----+----+----+----+----+----+----+-----+-------+--------------------+-----+-----------+----+----+----------+-----------+----+----+----+----+----+----+----+----+-----+-------+--------------------+------+---------------+---------------+----+----------+-----------+----+----+----+----+----+----+----+----+--------------------+-----------+--------------------+----+--------------------+-----------+-----+----------+-----------+----+----+----+----+----+----+----+--------------------+--------------------+------------+--------------------+------+--------------------+-----------+-----+----------+

In [12]:
final_df = df_load.select(df_load._c0.alias('collision_id'), df_load._c1.alias('crash_date'), df_load._c2.alias('crash_time'), df_load._c3.alias('borough'), df_load._c4.alias('zip_code'), df_load._c5.alias(
    'latitude'), df_load._c6.alias('longitude'), df_load._c7.alias('on_street_name'), df_load._c9.alias('number_of_persons_injured'), df_load._c10.alias('number_of_persons_killed'), df_load._c19.alias('vehicle_type'))

In [15]:
final_df.coalesce(1).write.format("csv").save(
    "hdfs://hadoop:8020/csv-data/abc.csv")